# Setup

In [ ]:
import os, json, itertools

from openai import OpenAI
from IPython.display import display, Markdown

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('OPEN_AI')

In [ ]:
# Setup OpenAI API key in the os
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
# Connect to the OpenAI API
client = OpenAI()

# Define the Inputs

In [ ]:
# Define the model inputs
MODEL = "gpt-4.1"
MODEL_MINI = "gpt-4.1-mini"
TOOLS = [{ "type": "web_search" }]

In [ ]:
# Developer message definition
developer_message = """
You are an expert deep researcher.
You must provide complete and in-depth research to the user.
"""

In [ ]:
# Request the topic for research
topic = input("Please enter the topic that you'd like to research: ")
topic

Please enter the topic that you'd like to research: vibe coding


'vibe coding'

# LLMs Asks Questions

In [ ]:
prompt_to_clarify = f"""
Ask 5 numbered clarifying questions about the following topic: {topic}.
The goal of the questions is to understand the intended purpose of the research.
Reply only with the questions
"""

In [ ]:
# Get the OpenAI API to ask 5 clarifying questions
clarify = client.responses.create(
    model=MODEL_MINI,
    input=prompt_to_clarify,
    instructions=developer_message
)

In [ ]:
# Get the questions
questions = clarify.output[0].content[0].text.split("\n")
questions

['1. Are you referring to "vibe coding" as a programming methodology, a specific coding style, or a cultural/creative concept related to coding?  ',
 '2. What is the primary goal of your research on vibe coding—understanding its technical aspects, its impact on developer productivity, or its influence on team dynamics and collaboration?  ',
 '3. Are you interested in how vibe coding affects software quality, user experience, or project timelines?  ',
 '4. Do you want to explore the psychological or social factors involved in vibe coding, such as motivation, creativity, or communication within development teams?  ',
 '5. Are you focusing your research on a particular programming language, framework, or community where vibe coding is prominent?']

In [ ]:
# Ask each question to the user
answers = []
for question in questions:
  answer = input(f"{question}: ")
  answers.append(answer)

1. Are you referring to "vibe coding" as a programming methodology, a specific coding style, or a cultural/creative concept related to coding?  :  programming methodology
2. What is the primary goal of your research on vibe coding—understanding its technical aspects, its impact on developer productivity, or its influence on team dynamics and collaboration?  : understanding its technical aspects and its impact on developer productivity
3. Are you interested in how vibe coding affects software quality, user experience, or project timelines?  : user experience
4. Do you want to explore the psychological or social factors involved in vibe coding, such as motivation, creativity, or communication within development teams?  : no
5. Are you focusing your research on a particular programming language, framework, or community where vibe coding is prominent?: no, I am thinking more like a broad brush overview


# Web Search

## Goals and Queries

In [ ]:
# Write the prompt_goals prompt
prompt_goals = f"""
Using the user's {answers} to the {questions}, write a goal sentence, and 5 web search queries for the research about {topic}.
Output: A json list of the goal sentence and the 5 web search queries that will reach it.
Format: {{\"goal\": \"...\", \"queries\": [\"q1\", ....]}}
"""

In [ ]:
# Use the responses API
goal_and_queries = client.responses.create(
    model=MODEL,
    input=prompt_goals,
    previous_response_id=clarify.id,
    instructions=developer_message
)

In [ ]:
# Formatting and loading as JSON
plan = json.loads(goal_and_queries.output[0].content[0].text)

In [ ]:
print(plan["goal"])
print(plan["queries"])

To provide a broad overview of vibe coding as a programming methodology, with a focus on understanding its technical aspects and its impact on developer productivity and user experience.
['What is vibe coding as a programming methodology', 'Technical aspects of vibe coding in software development', 'How does vibe coding affect developer productivity', 'Impact of vibe coding on user experience in applications', 'Overview and benefits of vibe coding methodology']


In [ ]:
# Store the goal and queries
goal = plan["goal"]
queries = plan["queries"]

## Web Search

In [ ]:
# Print the first query
print(queries[0])

What is vibe coding as a programming methodology


In [ ]:
# Web search with the resposnes endpoint
web_search = client.responses.create(
    model=MODEL,
    input=f"search: {queries[0]}",
    previous_response_id=goal_and_queries.id,
    instructions=developer_message,
    tools=TOOLS
)

In [ ]:
# Investigate the output

# Retrieve the ID
web_search.output[1].id

'msg_687be39d1260819a927a324bcae7774908e3115e5c497b08'

In [ ]:
# Retrive the content
web_search.output[1].content[0].text

'Vibe coding is an emerging programming methodology that leverages artificial intelligence (AI) tools to generate code based on natural language prompts, allowing developers to focus on high-level design and functionality rather than manual coding. This approach fosters a more intuitive and conversational interaction between the developer and the AI, streamlining the software development process.\n\n**Key Characteristics of Vibe Coding:**\n\n1. **Natural Language Prompting:** Developers describe desired features or behaviors in everyday language, and AI tools interpret these prompts to generate corresponding code. For example, a developer might instruct, "Create a login form with rounded fields," and the AI would produce the necessary code. ([docs.vibe-coding-framework.com](https://docs.vibe-coding-framework.com/core-concepts/what-is-vibe-coding?utm_source=openai))\n\n2. **AI as a Collaborative Partner:** In this paradigm, AI functions similarly to a pair programmer, assisting in code 

In [ ]:
# Build a function for the web searches
def run_search(query):
  web_search = client.responses.create(
      model=MODEL,
      input=f"search: {query}",
      previous_response_id=goal_and_queries.id,
      instructions=developer_message,
      tools=TOOLS
  )

  return {
      "query": query,
      "resp_id": web_search.output[1].id,
      "research_output": web_search.output[1].content[0].text
      }


In [ ]:
# Test the function
run_search(queries[0])

{'query': 'What is vibe coding as a programming methodology',
 'resp_id': 'msg_687be50d54d8819a9dab30c0ec0aea3408e3115e5c497b08',
 'research_output': 'Vibe coding is an emerging programming methodology that leverages artificial intelligence (AI) tools to generate code based on natural language prompts, allowing developers to focus on high-level design and functionality rather than manual coding. Coined by Andrej Karpathy, former AI director at Tesla, vibe coding emphasizes a conversational, intuitive approach where developers describe desired outcomes, and AI systems produce the corresponding code. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Vibe_coding?utm_source=openai))\n\n**Key Characteristics of Vibe Coding:**\n\n1. **Natural Language Prompting:** Developers articulate their requirements in everyday language, such as "Create a login form with rounded fields," and AI interprets these prompts to generate the appropriate code. ([docs.vibe-coding-framework.com](https://docs.vibe

## Verification

Veryifying if we have enough information to reach the stated goal.

In [ ]:
# Create a function to evalute the outcome
def evaluate_responses(collected):
  review = client.responses.create(
      model=MODEL,
      input=[
          {"role": "developer", "content": f"Research goal: {goal}"},
          {"role": "assistant", "content": json.dumps(collected)},
          {"role": "user", "content": "Does this information fully satisfy the goal? Answer Yes or No only."}
          ],
      instructions=developer_message
  )
  return "yes" in review.output[0].content[0].text.lower()

In [ ]:
# Verifying the information
collected = []
for _ in itertools.count():
  for query in queries:
    collected.append(run_search(query))
  if evaluate_responses(collected):
    break

  # if no: perform 5 more questions
  more_searches = client.responses.create(
      model=MODEL,
      input=[
          {"role": "assistant", "content": f"Current data: {json.dumps(collected)}"},
          {"role": "user", "content": f"This has not me the goal: {goal}. Write 5 more web searches to achieve the goal"}
      ],
      instructions=developer_message,
      previous_response_id=goal_and_queries.id
  )

  queries = json.loads(more_searches.output[0].content[0].text)

In [ ]:
collected

[{'query': 'What is vibe coding as a programming methodology',
  'resp_id': 'msg_687be715f100819a9d5b06975a625b2408e3115e5c497b08',
  'research_output': 'Vibe coding is an emerging programming methodology that leverages artificial intelligence (AI) tools to generate code based on natural language prompts, allowing developers to focus on high-level design and functionality rather than manual coding. This approach fosters a more intuitive and creative development process, enabling rapid prototyping and experimentation.\n\n**Key Characteristics of Vibe Coding:**\n\n1. **Natural Language Prompting:** Developers describe desired features or behaviors in everyday language, and AI systems interpret these prompts to produce corresponding code. For example, a developer might instruct, "Create a login form with rounded fields," and the AI generates the appropriate code. ([docs.vibe-coding-framework.com](https://docs.vibe-coding-framework.com/core-concepts/what-is-vibe-coding?utm_source=openai))\

# Write the Final Report

In [ ]:
# Use the reponses endpoint to write the deep research report
report = client.responses.create(
    model=MODEL,
    input=[
        {"role": "developer", "content": (f"Write a complete and detailed report abouth the research goal: {goal}"
                                          "Cite sources inline using [n] and append a reference"
                                          "List mapping [n] to url")},
        {"role": "assistant", "content": json.dumps(collected)},
    ],
    instructions=developer_message
)

print("\n=== FINAL REPORT ===")
display(Markdown(report.output[0].content[0].text))


=== FINAL REPORT ===


# Research Report: Overview of Vibe Coding as a Programming Methodology

## Introduction

"Vibe coding" is a rapidly emerging programming methodology that leverages artificial intelligence (AI) for code generation based on natural language prompts. This approach represents a significant shift in the way software is conceptualized, developed, and iterated upon. The methodology is characterized by its use of AI tools as collaborative partners, enabling developers to focus on design, creativity, and user experience rather than writing code line-by-line. This report provides an in-depth overview of the technical aspects of vibe coding, its impact on developer productivity, and its effects on user experience.

---

## 1. What Is Vibe Coding?

Vibe coding is defined as a software development methodology where AI-powered tools facilitate, assist, or even generate code from natural, conversational language prompts. The term gained widespread attention after AI researchers and practitioners, including Andrej Karpathy, demonstrated its potential in 2025. In this paradigm, the developer provides intent or functional requirements in everyday language, and the AI generates the corresponding code, which the developer then iteratively refines with feedback and further prompts.

Key characteristics include:

- **Natural language prompting**: Developers describe required features or behaviors in plain language rather than structured programming syntax.  
- **AI as a coding partner**: The AI tool serves as an assistant or collaborator, generating code, suggesting improvements, and handling routine tasks.  
- **Creativity and flow**: Vibe coding supports "flow state" programming, where idea-to-implementation feedback cycles are significantly shortened and creative prototyping is emphasized.

Sources: [1][2][3]

---

## 2. Technical Aspects of Vibe Coding

**A. Natural Language Prompting**

- Developers articulate functionalities or requirements using standard language (e.g., “Create a login form with Google authentication”).
- AI models interpret intent, often leveraging large language models (LLMs) or domain-specific AI systems, to generate, refine, and debug code.

**B. AI Integration and Iterative Refinement**

- Modern vibe coding workflows utilize cloud-based tools like GitHub Copilot, Replit, or custom LLM integrations.
- Developers interact with the AI in a conversational loop (`prompt → code generation → review → refined prompt`), blurring the line between human and machine contributors.

**C. Rapid Prototyping and Continuous Review**

- Quick feedback enables fast iteration, making vibe coding ideal for prototyping, proof-of-concept work, and small-to-medium projects.
- Human review remains vital for debugging, assessing code quality, and ensuring output matches real-world requirements.

**D. Code Quality, Security, and Documentation**

- While AI can automate many tasks, it may introduce bugs or security vulnerabilities if left unchecked. Human oversight, code review, and adherence to best practices are still critical.
- AI-generated code can lack structure or documentation, challenging maintainability for larger codebases or long-term projects.

Sources: [2][3][4][5][6]

---

## 3. Impact on Developer Productivity

### Benefits

- **Accelerated cycles:** Studies and industry reports suggest teams using AI-augmented methodologies report time savings. E.g., JPMorgan Chase cited a 10-20% engineering efficiency increase using AI tools ([7]).
- **Rapid prototyping:** Immediate translation of ideas into working code allows for more iterations in less time.
- **Higher code quality:** AI assistants provide instant syntax and logic checking, lowering error rates and suggesting best practices ([8]).
- **Reduced cognitive load:** Routine tasks are handled by AI, freeing developers for architectural and creative work ([9]).
- **Expanded access:** Non-traditional or less experienced developers can contribute more effectively, democratizing software creation ([10]).

### Challenges

- **Over-reliance risk:** Excessive dependence on AI can erode developers' understanding of the underlying codebase, increasing technical debt.
- **Quality, security, and maintenance:** The need for robust human oversight to maintain quality and mitigate risks is paramount, particularly for complex, mission-critical projects.

Sources: [3][7][8][9][10][11]

---

## 4. Impact on User Experience (UX)

### Positive Effects

- **User-centered applications:** By accelerating prototyping and iteration, developers can incorporate feedback quickly, leading to applications that better fit user needs and expectations ([12]).
- **Personalization:** Broader participation in development processes (including non-engineers) leads to more diverse, user-centered features ([10]).
- **Agility:** Fast development cycles allow applications to adapt rapidly to changing user requirements.

### Challenges

- **Code quality pitfalls:** AI-generated code can lack robustness, leading to hidden bugs, performance bottlenecks, or poor scalability ([13]).
- **Security vulnerabilities:** Large AI models may introduce insecure code due to limited context awareness ([14]).
- **Subjectivity and bias:** AI may perpetuate biases from training data, influencing the UX negatively for certain user groups ([15]).
- **Ethical concerns:** Use of AI-generated behavioral or emotional design can raise manipulation or privacy issues, especially if not transparent.

Sources: [12][13][14][15][16]

---

## 5. Overview and Benefits

In summary, vibe coding methodology offers:

- **Accelerated development and prototyping** for rapid experimentation and delivery.
- **Democratization of software creation,** making programming accessible to more people.
- **Enhanced creativity** and flow, supporting innovative and user-oriented solutions.
- **Improved teamwork** through collaborative and open communication fostered by AI tools.
- **Reduced burnout** and improved developer happiness, as routine work is shifted to the AI.

Caveats exist: over-reliance on AI, the need for careful human review, scalability issues, and ethical considerations mean this methodology is best suited (for now) to prototyping, innovation, and small-to-medium applications.

Sources: [3][4][5][9][11][16]

---

## References

1. [What is vibe coding? (Vibe Coding Framework Docs)](https://docs.vibe-coding-framework.com/core-concepts/what-is-vibe-coding?utm_source=openai)
2. [Vibe Coding - Wikipedia](https://en.wikipedia.org/wiki/Vibe_coding?utm_source=openai)
3. [Vibe Coding: Programar con Flow, Creatividad y Pasión (immune.institute)](https://immune.institute/en/blog/vibe-coding-programar-con-flow-creatividad-y-pasion/?utm_source=openai)
4. [Understanding the vibe coding trend and considerations for developers (TechRadar)](https://www.techradar.com/pro/understanding-the-vibe-coding-trend-and-considerations-for-developers?utm_source=openai)
5. [Vibe Coding Blog (binmile.com)](https://binmile.com/blog/vibe-coding/?utm_source=openai)
6. [The case against vibe coding (TheServerSide)](https://www.theserverside.com/tip/The-case-against-vibe-coding?utm_source=openai)
7. [Enhancing Team Productivity with Vibe Coding (LinkedIn)](https://www.linkedin.com/pulse/enhancing-team-productivity-vibe-coding-future-manav-shrivastava-oimoc?utm_source=openai)
8. [How vibe coding is enhancing developer productivity (Fueler.io)](https://fueler.io/blog/how-vibe-coding-is-enhancing-developer-productivity?utm_source=openai)
9. [Vibe Coding Mindset (gojive.dev)](https://gojive.dev/blog/static/vibe-coding-mindset?utm_source=openai)
10. ['Vibe coding' is the new DIY (FT.com)](https://www.ft.com/content/f4f3def2-2858-4239-a5ef-a92645577145?utm_source=openai)
11. [Vibe Coding Trends and Impact (Architech Today)](https://architech.today/vibe-coding-definition-trends-and-impact-in-modern-software-development/?utm_source=openai)
12. [Assessing user experience: How vibe coding affects application usability (Arsturn)](https://www.arsturn.com/blog/assessing-user-experience-how-vibe-coding-affects-application-usability?utm_source=openai)
13. [What’s Wrong with Vibe Coding? (Coaxsoft)](https://coaxsoft.com/blog/whats-wrong-with-vibe-coding?utm_source=openai)
14. [The Rise of Vibe Coding and Intuitive Software Development (ProfessionalDeveloper.net)](https://professionaldeveloper.net/2025/07/07/the-rise-of-vibe-coding-and-intuitive-software-development/?utm_source=openai)
15. [Vibe Coding: AI's Role in Modern Development (Clique8)](https://www.clique8.com/article/vibe-coding-ais-role-in-modern-development?utm_source=openai)
16. [Benefits and Challenges of Vibe Coding (Vibe Coding Framework Docs)](https://docs.vibe-coding-framework.com/benefits-and-challenges?utm_source=openai)

---

## Conclusion

Vibe coding is a transformative programming methodology bringing together AI and human creativity to reshape software development. It boosts productivity, democratizes application creation, and positions user experience at the center of the process. However, successful adoption requires a balanced approach that leverages AI's strengths while maintaining rigorous human oversight to ensure quality, security, and scalability.